# Experimentation

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import networkx as nx
from parse import *
from utils  import *
from pulp import *
from math import comb
import gurobipy as gp
from gurobipy import GRB

In [252]:
sample_input = pd.read_csv("samples/10.in", header = None)
sample_output = np.array(pd.read_csv("samples/10.out", header = None))

In [253]:
graph, budget = read_input_file("samples/10.in")

In [254]:
pos = nx.kamada_kawai_layout(graph)  # positions for all nodes
# nx.draw(graph, pos, with_labels=True)

In [255]:
# prob = LpProblem("proj",LpMaximize)

In [256]:
# # edges 
# # Add diagonal variables such that (1,1) have 0 happiness and sadness
# edges = np.array(["e01","e02","e03","e12","e13","e23"])
# weights = [9.2, 5.4, 2.123, 75.4,18,87]*3
# sad = [3,40.8,98,8,57.904,9.4]*3
# d = {}
# for j in range(3):
#     for i in edges:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# edges = list(d.values())
# # e11 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e22 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e33 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")

In [257]:
# # vertices 
# vertices = ["v0", "v1", "v2", "v3"]
# d = {}
# for j in range(3):
#     for i in vertices:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, cat="Integer")
# vertices = list(d.values())
# vertices

In [258]:
# prob += sum(edges[i] * weights[i] for i in range(len(weights)))

In [259]:
# prob += sum(edges[i] for i in [0,6,12]) == 1
# prob += sum(edges[i] for i in [1,7,13]) == 1
# prob += sum(edges[i] for i in [2,8,14]) == 1
# prob += sum(edges[i] for i in [3,9,15]) == 1
# prob += sum(edges[i] for i in [4,10,16]) == 1
# prob += sum(edges[i] for i in [5,11,17]) == 1

In [260]:
# prob += sum(edges[i] * sad[i] for i in range(6)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(6,12)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(12,18)) <= 42.314/3

# prob += sum(vertices[i] for i in [0,4,8]) == 1 
# prob += sum(vertices[i] for i in [1,5,9]) == 1
# prob += sum(vertices[i] for i in [2,6,10]) == 1
# prob += sum(vertices[i] for i in [3,7,11]) == 1


# # Linearization of quadratic variables (eijk =vik * vjk)

In [261]:
# prob

In [262]:
# prob.solve()
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# Scale  

Assumptions 
- Fewer rooms imply higher happiness scores in expectation 

In [5]:
sample_input = pd.read_csv("samples/10.in", header = None)
n = int(list(sample_input[0])[0])
s_max = float(list(sample_input[0])[1])
K = 3

In [6]:
# Parsing input data 
df = sample_input
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

In [7]:
i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

True

In [8]:
prob = LpProblem("solver",LpMaximize)

In [9]:
# edges 
d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = LpVariable("e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), cat="Binary")
edges_1 = list(d_e.values())


# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
len(edges_1) == comb(n,2)*K

True

In [11]:
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = LpVariable("v_{0}_{1}".format(i,j), cat="Binary")
vertices_1 = list(d_v.values())

# Create another edge array with different sort
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]

# Check
len(vertices_1) == n*K

True

In [12]:
# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, K)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,n-1):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

True


In [13]:
# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

True


## Naive Solver 

In [14]:
# Objective function 
prob += sum(edges_1[i] * happiness[i] for i in range(len(happiness)))

In [272]:
# # Not sure if we need this constraint now 
# # Constraint 1 
# constraint_2 = [sum(edges_1[i:i + K]) for i in range(0, len(edges_1), K)]
# for i in range(len(constraint_3)):
#     prob += constraint_2[i] == 1

In [15]:
# Constraint 2
# Each rooms has sadness levels <= s_max / K 
aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    prob += constraint_2[i] <= s_max / K

In [16]:
# Constraint 3
# Each person belongs to only one room
constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    prob += constraint_3[i] == 1

In [17]:
# Linearization of quadratic variables
# Constraint 4.1

for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big[i]
    
# Constraint 4.2
for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big_2[i]


# Constraint 4.3
for i in range(len(edges_1)):
    prob += edges_1[i] >= v_big[i] + v_big_2[i] - 1 
    
# # Constraint 4'
# for i in range(len(edges_1)):
#     prob += edges_1[i] == v_big[i] * v_big_2[i]

In [276]:
path = "/Users/smeetpatel/Library/gurobi.lic"
solver = GUROBI_CMD(path)

In [18]:
prob

solver:
MAXIMIZE
8.0*e_0_1_0 + 8.0*e_0_1_1 + 8.0*e_0_1_2 + 9.0*e_0_2_0 + 9.0*e_0_2_1 + 9.0*e_0_2_2 + 8.0*e_0_3_0 + 8.0*e_0_3_1 + 8.0*e_0_3_2 + 4.0*e_0_4_0 + 4.0*e_0_4_1 + 4.0*e_0_4_2 + 4.0*e_0_5_0 + 4.0*e_0_5_1 + 4.0*e_0_5_2 + 3.0*e_0_6_0 + 3.0*e_0_6_1 + 3.0*e_0_6_2 + 3.0*e_0_7_0 + 3.0*e_0_7_1 + 3.0*e_0_7_2 + 8.0*e_0_8_0 + 8.0*e_0_8_1 + 8.0*e_0_8_2 + 7.0*e_0_9_0 + 7.0*e_0_9_1 + 7.0*e_0_9_2 + 6.0*e_1_2_0 + 6.0*e_1_2_1 + 6.0*e_1_2_2 + 7.0*e_1_3_0 + 7.0*e_1_3_1 + 7.0*e_1_3_2 + 8.0*e_1_4_0 + 8.0*e_1_4_1 + 8.0*e_1_4_2 + 3.0*e_1_5_0 + 3.0*e_1_5_1 + 3.0*e_1_5_2 + 9.0*e_1_6_0 + 9.0*e_1_6_1 + 9.0*e_1_6_2 + 9.0*e_1_7_0 + 9.0*e_1_7_1 + 9.0*e_1_7_2 + 9.0*e_1_8_0 + 9.0*e_1_8_1 + 9.0*e_1_8_2 + 9.0*e_1_9_0 + 9.0*e_1_9_1 + 9.0*e_1_9_2 + 5.0*e_2_3_0 + 5.0*e_2_3_1 + 5.0*e_2_3_2 + 9.0*e_2_4_0 + 9.0*e_2_4_1 + 9.0*e_2_4_2 + 2.0*e_2_5_0 + 2.0*e_2_5_1 + 2.0*e_2_5_2 + 3.0*e_2_6_0 + 3.0*e_2_6_1 + 3.0*e_2_6_2 + 9.0*e_2_7_0 + 9.0*e_2_7_1 + 9.0*e_2_7_2 + 2.0*e_2_8_0 + 2.0*e_2_8_1 + 2.0*e_2_8_2 + 4.0*e_2_9_0 + 4.0

In [277]:
prob.solve(solver)

1

In [278]:
for v in prob.variables()[len(prob.variables()) - n*K : len(prob.variables())]:
    print(v.name, "=", v.varValue)

v_0_0 = 0.0
v_0_1 = 1.0
v_0_2 = 0.0
v_1_0 = 0.0
v_1_1 = 0.0
v_1_2 = 1.0
v_2_0 = 0.0
v_2_1 = 1.0
v_2_2 = 0.0
v_3_0 = 1.0
v_3_1 = 0.0
v_3_2 = 0.0
v_4_0 = 1.0
v_4_1 = 0.0
v_4_2 = 0.0
v_5_0 = 0.0
v_5_1 = 1.0
v_5_2 = 0.0
v_6_0 = 1.0
v_6_1 = 0.0
v_6_2 = 0.0
v_7_0 = 0.0
v_7_1 = 0.0
v_7_2 = 1.0
v_8_0 = 1.0
v_8_1 = 0.0
v_8_2 = 0.0
v_9_0 = 0.0
v_9_1 = 0.0
v_9_2 = 1.0


In [279]:
pairs = prob.variables()[0: len(prob.variables()) - n*K]
people_values = [v.varValue for v in pairs] 
np.sum(np.multiply(people_values, happiness))

75.0

In [280]:
sum(people_values)

12.0

# Gurobi

In [4]:
### Continuous 

inputs = pd.read_csv("phase1/20.in", header = None)
n = int(list(inputs[0])[0])
s_max = float(list(inputs[0])[1])
K = 5

# Parsing input data 
df = inputs
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

m = gp.Model("proj")
m.setParam('OutputFlag', 0)

# Some stuff

d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = m.addVar(name= "e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), vtype = GRB.BINARY)
edges_1 = list(d_e.values())

# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
print(len(edges_1) == comb(n,2)*K)
     
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = m.addVar(name = "v_{0}_{1}".format(i,j),  vtype = GRB.BINARY)
vertices_1 = list(d_v.values())

# Create another edge array with different sort
# vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys())]

# Check
print(len(vertices_1) == n*K)

# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, K)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,n-1):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

m.setObjective(sum(edges_1[i] * happiness[i] for i in range(len(happiness))), GRB.MAXIMIZE)

aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    m.addConstr(constraint_2[i] <=  s_max / K)

constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    m.addConstr(constraint_3[i] == 1)

# Constraint 4'
for i in range(len(edges_1)):
    m.addConstr(edges_1[i] == v_big[i] * v_big_2[i])

hardlimit = 30
m.setParam('TimeLimit', hardlimit)
m.optimize()

pairs = m.getVars()[0 : len(m.getVars()) - n*K]
people = m.getVars()[len(m.getVars()) - n*K : len(m.getVars())]
people_values = [v.X for v in people]
people_values = np.array_split(people_values, K)
pair_values = [v.X for v in pairs]
np.sum(np.multiply(pair_values, happiness))

output_dict = {}
for k in range(K):
    arr = []
    for i in range(n):
        if people_values[k][i] == 1: 
            arr += [i]
            output_dict[k] = arr

output_dict

True
True
True
True


{0: [0, 8, 10, 14],
 1: [1, 2, 12, 13],
 2: [3, 4, 11, 17],
 3: [5, 6, 9, 19],
 4: [7, 15, 16, 18]}

In [18]:
people

[<gurobi.Var v_0_0 (value 1.0)>,
 <gurobi.Var v_1_0 (value 0.0)>,
 <gurobi.Var v_2_0 (value -0.0)>,
 <gurobi.Var v_3_0 (value -0.0)>,
 <gurobi.Var v_4_0 (value 0.0)>,
 <gurobi.Var v_5_0 (value 0.0)>,
 <gurobi.Var v_6_0 (value -0.0)>,
 <gurobi.Var v_7_0 (value 1.0)>,
 <gurobi.Var v_8_0 (value 0.0)>,
 <gurobi.Var v_9_0 (value 0.0)>,
 <gurobi.Var v_10_0 (value 0.0)>,
 <gurobi.Var v_11_0 (value -0.0)>,
 <gurobi.Var v_12_0 (value 0.0)>,
 <gurobi.Var v_13_0 (value 1.0)>,
 <gurobi.Var v_14_0 (value 0.0)>,
 <gurobi.Var v_15_0 (value 1.0)>,
 <gurobi.Var v_16_0 (value 0.0)>,
 <gurobi.Var v_17_0 (value -0.0)>,
 <gurobi.Var v_18_0 (value -0.0)>,
 <gurobi.Var v_19_0 (value 0.0)>,
 <gurobi.Var v_0_1 (value -0.0)>,
 <gurobi.Var v_1_1 (value 0.0)>,
 <gurobi.Var v_2_1 (value 1.0)>,
 <gurobi.Var v_3_1 (value -0.0)>,
 <gurobi.Var v_4_1 (value 0.0)>,
 <gurobi.Var v_5_1 (value 1.0)>,
 <gurobi.Var v_6_1 (value -0.0)>,
 <gurobi.Var v_7_1 (value 0.0)>,
 <gurobi.Var v_8_1 (value -0.0)>,
 <gurobi.Var v_9_1 (val

In [7]:
inputs = pd.read_csv("phase1/test10.in", header = None)
float(list(inputs[0])[1])

99.0